# Control Systems 1, NB02: Introduction to System Classification and Properties
2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli


This interactive notebook is designed to introduce fundamental concepts in control systems engineering. It covers the basics of system modeling, classification, and analysis, with a focus on building intuition through visual and interactive examples.

Authors:
- Mark Benazet Castells; mbenazet@ethz.ch
- Felix Muller; fmuller@ethz.ch

## Learning Objectives

After completing this notebook, you should be able to:

1. Understand input-output models of linear time-invariant (LTI) systems.
2. Write the model of an LTI system with A, B, C, D matrices.
3. Understand the concept of "state" to model a system.
4. Classify systems as static / dynamic, causal/non-causal, time-invariant/time-varying, linear/nonlinear.
5. Understand the notion of equilibrium points.
6. Approximate a non-linear system with a linear one.

# Setup

## Installing the required packages:
The following packages are required to run this notebook. If you haven't installed them yet, you can do so by running the following cell.

In [ ]:
%pip install numpy matplotlib scipy ipywidgets

## Import the packages:
The following cell imports the required packages. Run it before running the rest of the notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from ipywidgets import interact, interactive, widgets, FloatSlider, Dropdown
from IPython.display import display

# 1. Input-Output Models

An input-output model is a map Σ from an input signal u(t) to an output signal y(t):

$$y = \Sigma u$$

or more explicitly:

$$y(t) = (\Sigma u)(t), \quad \forall t \in T$$

This can be represented by a block diagram:

<div style="text-align:center;">
    <img src="./img/block_dgm.png" alt="Block Diagram" width="500">
</div>

Depending on how Σ affects u, we classify the system in different ways:
- Linear vs. Nonlinear
- Causal vs. Non-causal
- Static (memoryless) vs. Dynamic
- Time-invariant vs. Time-varying

# 2. Classification of Systems

## 1. Linear vs. Nonlinear Systems

A system is linear if it satisfies the superposition principle:

$\Sigma(a u_1 + b u_2) = a\Sigma(u_1) + b\Sigma(u_2)$

Or in other words:

$y(a u_1 + b u_2) = a y(u_1) + b y(u_2)$


Where Σ represents the system, u₁ and u₂ are input signals, and a and b are scalars.

Use the interactive plot below to check this property for different systems:

- Linear system: $y = u$
- Nonlinear system: $y = u^2$
- Nonlinear system: $y = \sin(u)$
- Nonlinear + Linear system: $y = \sin(u) + u$

Parameters:
- $a, b$: Scalars for the superposition principle

Adjust the parameters and observe:
- If the solid and dashed lines overlap for all inputs, the system is linear.
- If they diverge, the system is nonlinear.

In [ ]:
def system_response(system_type, u):
    if system_type == 'Linear':
        return u
    elif system_type == 'Nonlinear (Quadratic)':
        return u**2
    elif system_type == 'Nonlinear (Sine)':
        return np.sin(u)
    elif system_type == 'Nonlinear + Linear (Sine + Linear)':
        return np.sin(u) + u

def check_linearity(system_type, a, b):
    u1 = np.linspace(-2, 2, 200)
    u2 = np.linspace(-2, 2, 200)
    
    # Individual responses
    y1 = system_response(system_type, u1)
    y2 = system_response(system_type, u2)
    
    # Linearity check
    y_sum = system_response(system_type, a*u1 + b*u2)
    y_individual_sum = a*y1 + b*y2
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Linearity plot
    ax.plot(u1, y_sum, label='Σ(au₁ + bu₂)')
    ax.plot(u1, y_individual_sum, label='aΣ(u₁) + bΣ(u₂)', linestyle='--')
    ax.set_title('Linearity Check')
    ax.set_xlabel('Input')
    ax.set_ylabel('Output')
    ax.legend()
    ax.grid(True)
    
    plt.tight_layout()
    plt.show()

# Create interactive widgets
system_dropdown = Dropdown(options=['Linear', 'Nonlinear (Quadratic)', 'Nonlinear (Sine)', 'Nonlinear + Linear (Sine + Linear)'], 
                           value='Linear', description='System:')
a_slider = FloatSlider(min=-2, max=2, step=0.1, value=1, description='a:')
b_slider = FloatSlider(min=-2, max=2, step=0.1, value=1, description='b:')

# Create interactive plot
interactive_plot = interactive(check_linearity, system_type=system_dropdown, a=a_slider, b=b_slider)

# Display interactive plot
display(interactive_plot)

## 2. Causal vs Non-Causal Systems

1. **Causal System**: A system where the output at any time $t$ depends only on the values of the input on ($-\infty, t$]. 
   In other words, the system doesn't respond to future inputs.

2. **Non-Causal System**: A system where the output at time $t$ can depend on future inputs (after time $t$). 
   Such systems are not physically realizable in "the real world" as it would depend on future information.

## 3. Static vs. Dynamic

1. **Static System**: A system where the output at time $t$ depends only on the input at time $t$.
   In other words, the system has no memory of past inputs.

2. **Dynamic System**: A system where the output at time $t$ depends on the input at time $t$ and past inputs.

A good example of a static system is a **volume knob on a speaker**. The volume level is always controlled by the current position of the knob, without any consideration of previous settings. No matter how much the knob was turned earlier, the current volume depends only on the knob's current position.

An example of a dynamic system is the **longitudinal speed of a car**. A specific depression of the gas pedal does not correspond to one specific speed, because the car's speed depends on both how much the gas pedal is currently pressed and how it has been pressed over time. Other factors such as inertia and friction also contribute, meaning the speed evolves dynamically based on the system's history.

## 4. Time-Invariant vs. Time-Varying

1. **Time-Invariant System**: A system where the input-output relationship does not change over time.
   In other words, the system's properties do not change with time.

2. **Time-Varying System**: A system where the input-output relationship changes over time.

Imagine a perfect actuator in a control system:

1. Ideal Motor: 
   This actuator consistently produces the same output for a given input, regardless of when the input is applied. It's like a brand-new, high-quality motor that responds identically today, tomorrow, or a year from now.

2. Aging Motor:
   In reality, actuators may degrade over time due to wear and tear. An aging motor might produce less output for the same input as time passes. This change in behavior over time characterizes a time-varying system.

<div style="text-align:center;">
    <img src="./img/actuator_decay.png" alt="Block Diagram" width="800">
</div>

# 3. State-Space Models

When a system is causal, we can transform it from any form that might come out of the modeling techniques you saw last week to a state-space model. In general, a state space model looks like this:

$$\dot{x}(t) = f(t, x(t), u(t))$$
$$y(t) = g(t, x(t), u(t))$$
$$x(t_0) = x_0$$

Where:
- $x(t)$ is the state of the system.
- $u(t)$ is the input.
- $y(t)$ is the output.
- $x_0$ is the initial condition.

In this course, we will study only Linear Time-Invariant (LTI) systems. These systems are linear and time-invariant, which means they satisfy the superposition principle and their properties do not change over time. There are many useful techniques that we can't use for nonlinear systems that work great for LTI systems.

In the LTI case, the state-space model can be written as:

$$\dot{x}(t) = Ax(t) + Bu(t)$$
$$y(t) = Cx(t) + Du(t)$$
$$x(t_0) = x_0$$

Where:
- $A, B, C, D$ are matrices that define the system.

Most Systems are not LTI, but many systems can be approximated very well by LTI models. On the [graph above](#1-linear-vs-nonlinear-systems), you can see that $\Sigma(a u_1 + b u_2)$ and $a\Sigma(u_1) + b\Sigma(u_2)$ are very similar in some spots for the nonlinear systems, mainly the origin. We take advantage of that by **linearizing** nonlinear systems around these spots.

These spots, where a linearization is valid and useful, are called equilibrium points. For the general nonlinear state-space model, as seen model above, the equilibrium point $(x_e, u_e)$ is defined with:

$$f(x_e, u_e) = 0$$

This means that if $x_e$ is specified we can calculate $u_e$, and vice-versa. It's possible for a system to have multiple equilibrium points.

## 4. Linearization of a Nonlinear System

In general, a nonlinear system can be approximated by a linear model through a Taylor series expansion around its equilibrium point. The linearization process, in its most basic form, follows this structure:

### Given:
$$
\begin{cases}
\dot{x}(t) = f(x(t), u(t)) \\
y(t) = g(x(t), u(t))
\end{cases}
$$

### Wanted:
$$
\begin{cases}
\dot{x}(t) = A x(t) + B u(t) \\
y(t) = C x(t) + D u(t)
\end{cases}
$$

### Equilibrium point: 
$$(x_e, u_e)$$

### Linearization:

$$
A = \begin{bmatrix}
\frac{\partial f_1}{\partial x_1} & \cdots & \frac{\partial f_1}{\partial x_n} \\
\vdots & \ddots & \vdots \\
\frac{\partial f_n}{\partial x_1} & \cdots & \frac{\partial f_n}{\partial x_n}
\end{bmatrix}_{x_e, u_e}
$$

$$
B = \begin{bmatrix}
\frac{\partial f_1}{\partial u_1} \\
\vdots \\
\frac{\partial f_n}{\partial u_1}
\end{bmatrix}_{x_e, u_e}
$$

$$
C = \begin{bmatrix}
\frac{\partial g_1}{\partial x_1} & \cdots & \frac{\partial g_1}{\partial x_n} 
\end{bmatrix}_{x_e, u_e}
$$

$$
D = \begin{bmatrix}
\frac{\partial g_1}{\partial u_1}
\end{bmatrix}_{x_e, u_e}
$$


It is important to note that in the resulting linear system, $x(t)$, $u(t)$ and $y(t)$ are not the original modeling coordinates, but rather represent the deviations from the equilibrium point.

### Example:

We now explore the linearization of an example nonlinear system around an equilibrium point. This system is not a practical control example, but facilitates showing the effect of linearization visually. The system we analyze is the nonlinear function:

$$
f(x) = \sin(x)
$$

We will take a more intuitive approach, but the general approach would of course also work. Later in this notebook you will see an example using the general approach.

First we have to find an equilibrium point, in this case, we choose the equilibrium point $x = \pi$, where $\sin(\pi) = 0$.

The linear approximation of a simple nonlinear function is given by the tangent line at the equilibrium point. To find this, we calculate the derivative of $f(x)$. For $f(x) = \sin(x)$, the derivative is:

$$
f'(x) = \cos(x)
$$

At $x = \pi$, the slope of the tangent line is:

$$
f'(\pi) = \cos(\pi) = -1
$$

Thus, the linear approximation of the system near $x = \pi$ is:

$$
y = -1 \cdot (x - \pi)
$$

### Visualization

In the visualization, you can see the nonlinear function as well as the linearization at the point $x_0$​ (adjustable with the slider). When you zoom in, meaning you see the area close to the linearization point, there is almost no difference between both functions when $x_0 = \pi$, as it's the equilibrium point. However, if you zoom out, you see that both functions begin to differ by a lot far from the equilibrium point.
The time-domain plot shows how this difference affects the system's behavior over time. The blue line represents the nonlinear system, while the red dashed line shows the linearized system.
This effect also exists for the linearization of control systems in general and must always be considered. The linearization of a system is only valid close to the equilibrium point around which the system was linearized.

In [ ]:
# Nonlinear system
def nonlinearized_system(X, t):
    x, y = X
    dx = y
    dy = -np.sin(x)
    return [dx, dy]

# Linearized system
def linearized_system(X, t, x0):
    x, y = X
    dx = y
    dy = -np.cos(x0) * (x - x0)
    return [dx, dy]

# Solve and plot time domain
def solve_and_plot_time(x0):
    t = np.linspace(0, 10, 1000)
    X0 = [0.1, 0]
    
    # Nonlinear solution
    sol_nonlinear = odeint(nonlinearized_system, X0, t)
    
    # Linearized solution
    sol_linear = odeint(linearized_system, X0, t, args=(x0,))
    
    plt.figure(figsize=(12, 6))
    plt.plot(t, sol_nonlinear[:, 0], label='Nonlinear')
    plt.plot(t, sol_linear[:, 0], label='Linearized')
    plt.xlabel('Time')
    plt.ylabel('x')
    plt.title(f'System behavior (linearized at x0 = {x0:.2f})')
    plt.legend()
    plt.grid(True)
    plt.show()

# Define the nonlinear system f(x) = sin(x)
def nonlinear_function(x):
    return np.sin(x)

# Calculate the linear approximation around the equilibrium point
def linear_approximation(x, equilibrium_point):
    slope = np.cos(equilibrium_point)
    return slope * (x - equilibrium_point) + np.sin(equilibrium_point)

# Plot the nonlinear system and its linearization
def plot_system(zoom, x0):
    # Generate x values based on zoom level
    x_vals = np.linspace(x0 - zoom, x0 + zoom, 400)
    
    # Nonlinear system output
    y_nonlinear = nonlinear_function(x_vals)
    
    # Linear approximation at equilibrium point x0
    y_linear = linear_approximation(x_vals, x0)
    
    # Plot the results
    plt.figure(figsize=(12, 6))
    
    # Plot the nonlinear system
    plt.plot(x_vals, y_nonlinear, label=r'Nonlinear System: $f(x) = \sin(x)$', color='blue')
    
    # Plot the linear approximation (tangent line)
    plt.plot(x_vals, y_linear, label=f'Linearization at x={x0:.2f}', linestyle='--', color='red')
    
    # Highlight the equilibrium point
    plt.scatter(x0, nonlinear_function(x0), color='black', zorder=5)
    plt.text(x0 + 0.1, np.sin(x0), f'Linearization Point ({x0:.2f}, {np.sin(x0):.2f})', fontsize=12)
    
    # Set plot details
    plt.title(f'Nonlinear System and Linearization around x = {x0:.2f}')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend()
    plt.grid(True)
    
    # Adjust x and y limits to keep linearization point in the middle
    plt.xlim(x0 - zoom, x0 + zoom)
    y_center = nonlinear_function(x0)
    plt.ylim(y_center - zoom, y_center + zoom)
    
    plt.show()

# Combined interactive function
def interactive_analysis(x0, zoom):
    plot_system(zoom, x0)
    solve_and_plot_time(x0)
    

# Create interactive widgets
interactive_plot = interactive(
    interactive_analysis, 
    x0=widgets.FloatSlider(min=-np.pi, max=np.pi, step=0.01, value=0, description='x0:'),
    zoom=widgets.FloatSlider(min=0.1, max=4.0, step=0.1, value=2.0, description='Zoom:')
)
display(interactive_plot)

We can clearly see that if the linearization is done around the equilibrium point, the linear approximation is very good in the vicinity of the equilibrium point. However, as we move further away from the equilibrium point, the linear approximation becomes less accurate and the linear model deviates greatly from the nonlinear one, even becoming unstable.

# 5. PS2 Exercise 5

Consider a pendulum that is mounted to a wall and connected to a horizontal spring as shown in Figure \ref{fig:eng_diag}.
Let $l$ be the length of the pendulum, $J$ its moment of inertia, $m$ its mass, and let $\lambda$ represent the damping constant of the pendulum acting at the pivot (damped rotation due to friction).

Further, let $k$ represent the spring constant of the spring, and denote by $a$ the distance to the pivot of the point that connects the spring and the pendulum. Assume the spring to be relaxed at $\varphi =0$.
The system is actuated by an external force $F(t)$ which acts at a right angle to the pendulum.
There is a sensor measuring the angle $\varphi$ which we assume to be limited to $\varphi(t)\in\left(-\frac{\pi}{2},\frac{\pi}{2}\right)$.

<div style="text-align:center;">
    <img src="./img/pendel.png" alt="Block Diagram" width="400">
</div>

## System Dynamics

The differential equation governing the dynamics of the above described system is assumed to be given by

$$J  \cdot\ddot{\varphi}(t) = -l \cdot m \cdot g  \cdot \sin \varphi(t) - \frac{a^2 \cdot k}{2} \cdot \sin 2\varphi(t) - \lambda \cdot \dot{\varphi}(t) + l\cdot F(t).$$

Further, let the system parameters be
$$
\begin{array}{rcl}
l &=& 1\text{ m}\\[0.2em]
m &=& 1\text{ kg}\\[0.2em]
g &=& 10\text{ m/s}^2\\[0.2em]
a &=& 0.5\text{ m}\\[0.2em]
k &=& 10\text{ N/m}\\[0.2em]
\lambda &=& 3\text{ Nms/rad}\\[0.2em]
J &=& 1\text{ Nms}^2\text{/rad}\\[0.2em]
\end{array}
$$
The initial conditions at $t=0$ are assumed to be such that pendulum is at angle $\varphi=0$ with angular velocity $\dot \varphi =0$.

## Nonlinear System Representation

Choosing the state vector $x(t) = \begin{bmatrix} x_1(t) \\ x_2(t) \end{bmatrix} = \begin{bmatrix} \varphi(t) \\ \dot \varphi(t) \end{bmatrix}$, the input vector $u(t) = F(t)$, and the output vector $y(t) = \varphi(t)$, the system can be represented in state-space form as follows:

$$
\begin{array}{rcl}
\dot{x}(t) &=& f(x(t),u(t))\\
y(t) &=& g(x(t),u(t))
\end{array}
$$

For our specific case, this becomes:

$$
\begin{array}{rcl}
\dot{x}(t) &=& \begin{bmatrix} x_2(t) \\ \frac{-l \cdot m \cdot g  \cdot \sin x_1(t) - \frac{a^2 \cdot k}{2} \cdot \sin 2x_1(t) - \lambda \cdot x_2(t) + l\cdot u(t)}{J} \end{bmatrix}\\
y(t) &=& x_1(t)
\end{array}
$$

## Linearizing the System

The system is nonlinear, but we can linearize it around an equilibrium point. In this case, we will linearize it around the equilibrium point $\varphi = 0$, $\dot \varphi = 0$ and $F(t) = 0$.

The linearized system is given by:

$$
\begin{array}{rcl}
\dot{x}(t) &=& \begin{bmatrix} 0 & 1 \\ -\frac{l \cdot m \cdot g + a^2 \cdot k}{J} & -\frac{\lambda}{J} \end{bmatrix} x(t) + \begin{bmatrix} 0 \\ \frac{l}{J} \end{bmatrix} u(t) \\[0.5em]
y(t) &=& \begin{bmatrix} 1 & 0 \end{bmatrix} x(t)
\end{array}
$$


## Task

We'll simulate a pendulum system with a control input $F(t)$ that jumps from $0 N$ to $0.1 N$ at $10$ seconds, then to $5 N$ at $15$ seconds. Our Python script will model this input and simulate the system's response over $20$ seconds, plotting the results to verify the behavior. We will also see how the linearized system behaves compared to the nonlinear one.

You can also change how much $F_1$ is applied at $t_1$ and how much $F_2$ is applied at $t_2$ and the initial conditions to see how the system behaves.

In [ ]:
# System parameters
l = 1.0  # m
m = 1.0  # kg
g = 10.0  # m/s^2
a = 0.5  # m
k = 10.0  # N/m
lambda_ = 3.0  # Nms/rad
J = 1.0  # Nms^2/rad

# Control input F(t)
def F(t, F1, F2, t1, t2):
    if t < t1:
        return 0
    elif t < t2:
        return F1
    else:
        return F2

# Nonlinear system dynamics
def nonlinear_system(x, t, F1, F2, t1, t2):
    phi, phi_dot = x
    F_t = F(t, F1, F2, t1, t2)
    phi_ddot = (-l * m * g * np.sin(phi) - 0.5 * a**2 * k * np.sin(2*phi) - lambda_ * phi_dot + l * F_t) / J
    return [phi_dot, phi_ddot]

# Linear system dynamics
def linear_system(x, t, F1, F2, t1, t2):
    phi, phi_dot = x
    F_t = F(t, F1, F2, t1, t2)
    phi_ddot = (-l * m * g * phi - a**2 * k * phi - lambda_ * phi_dot + l * F_t) / J
    return [phi_dot, phi_ddot]

# Simulation and plotting function
def simulate_and_plot(F1, F2, t1, t2, phi0, phi_dot0):
    # Time array
    t = np.linspace(0, 20, 1000)

    # Initial conditions
    x0 = [phi0, phi_dot0]

    # Solve ODEs
    nonlinear_solution = odeint(nonlinear_system, x0, t, args=(F1, F2, t1, t2))
    linear_solution = odeint(linear_system, x0, t, args=(F1, F2, t1, t2))

    # Extract phi and phi_dot
    nonlinear_phi = nonlinear_solution[:, 0]
    linear_phi = linear_solution[:, 0]

    # Calculate F(t) for plotting
    F_t = np.array([F(ti, F1, F2, t1, t2) for ti in t])

    # Plotting
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle('Pendulum System Simulation: Nonlinear vs Linear', fontsize=16, y=1.05)

    # Plot phi
    ax1.plot(t, nonlinear_phi, label='Nonlinear')
    ax1.plot(t, linear_phi, label='Linear', linestyle='--')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Angle (rad)')
    ax1.set_title('Pendulum Angle (φ)')
    ax1.grid(True)
    ax1.legend()

    # Plot F(t)
    ax2.plot(t, F_t)
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Force (N)')
    ax2.set_title('Control Input F(t)')
    ax2.grid(True)

    plt.tight_layout()
    plt.subplots_adjust(top=0.85, wspace=0.3)
    plt.show()

# Create widgets
F1_slider = widgets.FloatSlider(value=0.1, min=0, max=1, step=0.1, description='F1:')
F2_slider = widgets.FloatSlider(value=5, min=0, max=10, step=0.1, description='F2:')
t1_slider = widgets.FloatSlider(value=10, min=0, max=20, step=0.1, description='t1:')
t2_slider = widgets.FloatSlider(value=15, min=0, max=20, step=0.1, description='t2:')
phi0_slider = widgets.FloatSlider(value=0.0, min=-np.pi/2, max=np.pi/2, step=0.01, description='phi_0:')
phi_dot0_slider = widgets.FloatSlider(value=0.0, min=-5, max=5, step=0.01, description='dphi_0:')

# Create interactive output
interactive_plot = widgets.interactive(simulate_and_plot, 
                                       F1=F1_slider, F2=F2_slider, 
                                       t1=t1_slider, t2=t2_slider,
                                       phi0=phi0_slider, phi_dot0=phi_dot0_slider)

# Display the interactive plot
display(interactive_plot)